In [1]:
import pickle
import numpy as np
from classes import classes, colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

In [3]:
colors_reg = ["#DB073D", "#DBA507", "#0D6986"]
fig = make_subplots(rows=2, cols=1,
                    # shared_yaxes=True,
                    vertical_spacing=0.2,
                    subplot_titles=("LPF Data",
                                    "Interpolated Data"))
matrix = 0
sensor = 0
num_samples = 3
time_offset = 20
time_offset_end = time_offset + num_samples
starts = []
ends = []
for heater_idx in range(3):
    heater_data = sensor_data[f"mat_{matrix}"][sensor][heater_idx]
    time_arr = heater_data["Time Since PowerOn"].values[time_offset:time_offset_end]
    starts.append(time_arr[0])
    ends.append(time_arr[-1])
    fig.add_trace(go.Scatter(x=time_arr,
                             y=heater_data["Filtered"].values[time_offset:time_offset_end],
                             mode="lines+markers",
                             marker_color=colors_reg[heater_idx],
                             name=f"H.S. {heater_idx}",
                             showlegend=False,
                             legendgroup=f"H.S. {heater_idx}"),
                  row=1, col=1)

start = min(starts)
end = min(ends)
num_samples = 6
time_arr = np.linspace(start, end, num_samples)
for heater_idx in range(3):
    heater_data = sensor_data[f"mat_{matrix}"][sensor][heater_idx]
    interp_resistance = interp_funcs[f"mat_{
        matrix}"][sensor][heater_idx](time_arr)
    fig.add_trace(go.Scatter(x=time_arr,
                             y=interp_resistance,
                             mode="lines+markers",
                             marker_color=colors_reg[heater_idx],
                             name=f"H.S. {heater_idx}",
                             showlegend=False,
                             legendgroup=f"H.S. {heater_idx}"),
                  row=2, col=1)

fig.update_yaxes(title_text="Resistance (Ohms)", row=1,
                 col=1, tickformat=".0e")
fig.update_yaxes(title_text="Resistance (Ohms)", row=2,
                 col=1, tickformat=".0e")

fig.update_xaxes(title_text="Time since power on (ms)",
                 row=1, col=1, tickformat=".0e")
fig.update_xaxes(title_text="Time since power on (ms)",
                 row=2, col=1, tickformat=".0e")

fig.update_layout(height=500, width=700,
                  title_x=0.5,
                  font_family="Times New Roman",
                  title_font_family="Times New Roman",
                  title=dict(text="Effects of Interpolation"),
                  margin=dict(t=50, r=10, b=0, l=0))
# fig.write_image(f"interp.pdf")
fig.show()

In [15]:
def plot_sensor_it(matrix, sensor_idx, sensor_data, labels, classes, colors, save_pdf=False, include_labels=False):
    fig = go.Figure()

    sensors = sensor_data[f"mat_{matrix}"]
    for profile_idx in sensors[sensor_idx]:
        fig.add_trace(go.Scatter(x=sensors[sensor_idx][profile_idx]["Time Since PowerOn"].values,
                                 y=sensors[sensor_idx][profile_idx]["Resistance Gassensor"].values,
                                 mode="lines",
                                 showlegend=False,
                                 name=f"H.S. {profile_idx}"))

    if include_labels:
        for label in labels[f"mat_{matrix}"]:
            fig.add_vrect(x0=label["start"],
                          x1=label["end"],
                          annotation_text=classes[label["label"]],
                          annotation_position="top left",
                          fillcolor=colors[label["label"]],
                          opacity=0.2,
                          line_width=0)

    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")

    title = f"Matrix {matrix}, Sensor {sensor_idx}"
    fig.update_layout(height=300, width=700,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    if save_pdf:
        fig.write_image(f"figures/{matrix}_{sensor_idx}_labels.pdf")
    fig.show()

In [17]:
plot_sensor_it(0, 6,
               sensor_data, labels,
               classes, colors,
               save_pdf=True,
               include_labels=True)